In [1]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
import numpy as np

def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    elif epoch > 100:
        lrate = 0.0003        
    return lrate

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Input to z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

weight_decay = 1e-4
model = Sequential()

# Hidden Layer 1
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), kernel_initializer='orthogonal', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

# Hidden Layer 2
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), kernel_initializer='orthogonal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Hidden layer 3
model.add(Conv2D(256, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), kernel_initializer='orthogonal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.summary()

#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=[0.9, 1.10]
    )
datagen.fit(x_train)

#training
batch_size = 64

mc = ModelCheckpoint('model6_weights_{epoch:08d}.h5', 
                                     save_weights_only=True, period=5)

opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), shuffle=True,\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=125,\
                    verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule), mc])

#save to disk
model_json = model.to_json()
with open('model6.json', 'w') as json_file:
    json_file.write(model_json)

model.save_weights('model6.h5')    

#testing
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
activation_2 (Activation)    (None, 16, 16, 128)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 128)       512       
__________

781/781 [==============================] - 20s 25ms/step - loss: 0.5322 - acc: 0.8539 - val_loss: 0.5173 - val_acc: 0.8634
Epoch 104/125
781/781 [==============================] - 20s 25ms/step - loss: 0.5351 - acc: 0.8531 - val_loss: 0.5255 - val_acc: 0.8615
Epoch 105/125
781/781 [==============================] - 19s 25ms/step - loss: 0.5316 - acc: 0.8551 - val_loss: 0.5612 - val_acc: 0.8537
Epoch 106/125
781/781 [==============================] - 20s 25ms/step - loss: 0.5269 - acc: 0.8579 - val_loss: 0.5829 - val_acc: 0.8507
Epoch 107/125
781/781 [==============================] - 20s 25ms/step - loss: 0.5333 - acc: 0.8551 - val_loss: 0.5241 - val_acc: 0.8649
Epoch 108/125
781/781 [==============================] - 20s 26ms/step - loss: 0.5313 - acc: 0.8564 - val_loss: 0.5536 - val_acc: 0.8504
Epoch 109/125
781/781 [==============================] - 20s 26ms/step - loss: 0.5273 - acc: 0.8571 - val_loss: 0.5250 - val_acc: 0.8627
Epoch 110/125
781/781 [==============================] 

In [2]:
score = model.evaluate(x_test, y_test, batch_size=128, verbose=0)
print(model.metrics_names)
print(score)

['loss', 'acc']
[0.5191707515716553, 0.8603]
